In [1]:
import requests
import pandas as pd
import numpy as np
import json

In [ ]:
import json

import urllib.request

def make_request_urllib(url):
    try:
        with urllib.request.urlopen(url) as response:
            data = response.read()
            return json.loads(data.decode('utf-8'))
    except urllib.error.URLError as e:
        print(f"An error occurred: {e}")
        return None

In [2]:
def make_request(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
        return json.loads(response.text)
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None
    
#main table of responses
response = make_request("https://gbfs.capitalbikeshare.com/gbfs/2.3/gbfs.json")['data']['en']

In [11]:
#station information
stations = make_request("https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/station_information.json")['data']['stations']
#station region
station_regions = make_request('https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/system_regions.json')['data']['regions']
#json to dataframe
df = pd.DataFrame(stations)
df_region = pd.DataFrame(station_regions)
#merge dataframes
df_region = df_region.rename(columns = {'name':'region_name'})

df = pd.merge(df, df_region, left_on='region_id', right_on='region_id', how='left')

#convert lat and lon to float
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

#write data to csv
df.to_csv('stations.csv', index=False)
df.sort_values(by=['lat', 'lon'], inplace=True)
df

,name,region_id,capacity,lon,short_name,station_id,rental_uris,lat,region_name
706,Charles Arrington Dr & Manchester Lakes Dr,NaN,12,-77.150359,32290,1917259365558921792,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",38.766844,NaN
118,Franconia-Springfield Metro North,NaN,19,-77.168902,32289,1917259666206632502,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",38.766946,NaN
406,Metro Park Dr & Walker Ln,NaN,12,-77.156504,32296,1957669092295526720,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",38.768430,NaN
166,Seatrend Way & Topsails Ln,NaN,12,-77.160014,32288,1917259953969440770,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",38.775639,NaN
419,Fleet St & Waterfront St,133,15,-77.016059,32406,d4e8307d-5fb2-4ece-a66a-1522b1ebf79c,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",38.782633,Prince George's County
...,...,...,...,...,...,...,...,...,...
129,Shady Grove Metro West,43,19,-77.166093,32045,0825e56c-1f3f-11e7-bf6b-3863bb334450,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",39.119765,"Montgomery County, MD (North)"
603,Shady Grove Metro East,43,19,-77.164969,32067,08265563-1f3f-11e7-bf6b-3863bb334450,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",39.121327,"Montgomery County, MD (North)"
684,Needwood Rd & Eagles Head Ct,43,15,-77.157410,32018,0825962a-1f3f-11e7-bf6b-3863bb334450,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",39.123513,"Montgomery County, MD (North)"
654,Crabbs Branch Avenue & Gramercy Blvd,43,11,-77.164467,32095,2069782102977517916,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",39.124921,"Montgomery County, MD (North)"


In [15]:
#free bike status
free_bike_status = make_request("https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/free_bike_status.json")['data']['bikes']
#vehicle type
vehicle_types = make_request("https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/vehicle_types.json")['data']['vehicle_types']
#json to dataframe
df_free_bikes = pd.DataFrame(free_bike_status)
df_vehicle_types = pd.DataFrame(vehicle_types)
#join dataframes on vehicle_type_id
df_free_bikes = df_free_bikes.merge(df_vehicle_types, on='vehicle_type_id', how='left')
#clean up dataframe
#df_free_bikes = df_free_bikes[['bike_id', 'lat', 'lon']]
#convert lat and lon to float
#df_free_bikes['lat'] = df_free_bikes['lat'].astype(float)
df_free_bikes['lon'] = df_free_bikes['lon'].astype(float)


In [16]:
df_free_bikes

,lon,is_disabled,vehicle_type_id,lat,is_reserved,current_range_meters,bike_id,rental_uris,propulsion_type,form_factor,max_range_meters
0,-76.985174,0,2,38.865973,0,18990.2592,eb5b436c4bd87285702787d392060b23,{'android': 'https://dc.lft.to/lastmile_qr_sca...,electric_assist,bicycle,55844.24
1,-77.014902,0,2,38.975565,0,5471.7696,4d16bb5d121cc1ac6052e3ba552cfe0e,{'android': 'https://dc.lft.to/lastmile_qr_sca...,electric_assist,bicycle,55844.24
2,-77.030774,0,2,38.922470,0,12231.0144,91a7391de71db943b21684cbf3c4e830,{'android': 'https://dc.lft.to/lastmile_qr_sca...,electric_assist,bicycle,55844.24
3,-77.112705,0,2,39.071359,0,23013.6192,50e40042f6ac4fddbb9853454b00630d,{'android': 'https://dc.lft.to/lastmile_qr_sca...,electric_assist,bicycle,55844.24
4,-76.975639,0,2,39.046580,0,55522.3680,fd48f9ae8aaf12f7f20f320327017351,{'android': 'https://dc.lft.to/lastmile_qr_sca...,electric_assist,bicycle,55844.24
...,...,...,...,...,...,...,...,...,...,...,...
807,-77.115630,0,2,38.912775,0,29772.8640,30f6b77bb86154d31627ebcaa7ed64fe,{'android': 'https://dc.lft.to/lastmile_qr_sca...,electric_assist,bicycle,55844.24
808,-76.996178,0,2,38.971916,0,56970.7776,fabe7ee6eb08990aa8b351e234d67df4,{'android': 'https://dc.lft.to/lastmile_qr_sca...,electric_assist,bicycle,55844.24
809,-76.972772,0,2,38.992796,0,27036.9792,89f4735e0024f681443fd026d014e88f,{'android': 'https://dc.lft.to/lastmile_qr_sca...,electric_assist,bicycle,55844.24
810,-77.011757,0,2,38.970602,0,19633.9968,0b82882778a890dca0ec96ed69758004,{'android': 'https://dc.lft.to/lastmile_qr_sca...,electric_assist,bicycle,55844.24


In [4]:
vehicle_types

[{'propulsion_type': 'human',
  'form_factor': 'bicycle',
  'vehicle_type_id': '1'},
 {'propulsion_type': 'electric_assist',
  'max_range_meters': 55844.24,
  'form_factor': 'bicycle',
  'vehicle_type_id': '2'}]

In [9]:
free_bike_status = make_request("https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/free_bike_status.json")
#change key to timestamp
free_bike_status

{'data': {'bikes': [{'lon': -76.985162854,
    'is_disabled': 0,
    'vehicle_type_id': '2',
    'lat': 38.86598146,
    'is_reserved': 0,
    'current_range_meters': 18990.2592,
    'bike_id': 'dcc06775bf5af87d0be842c4182606e5',
    'rental_uris': {'android': 'https://dc.lft.to/lastmile_qr_scan',
     'ios': 'https://dc.lft.to/lastmile_qr_scan'}},
   {'lon': -77.015416622,
    'is_disabled': 0,
    'vehicle_type_id': '2',
    'lat': 38.974904656,
    'is_reserved': 0,
    'current_range_meters': 5471.7696,
    'bike_id': '5cd1e5f370f4cb336dfb498856707b19',
    'rental_uris': {'android': 'https://dc.lft.to/lastmile_qr_scan',
     'ios': 'https://dc.lft.to/lastmile_qr_scan'}},
   {'lon': -77.023230076,
    'is_disabled': 0,
    'vehicle_type_id': '2',
    'lat': 38.957342148,
    'is_reserved': 0,
    'current_range_meters': 24462.0288,
    'bike_id': 'fef8c8a3568d0e33981757f3e398effc',
    'rental_uris': {'android': 'https://dc.lft.to/lastmile_qr_scan',
     'ios': 'https://dc.lft.to/l

In [ ]:
import boto3
import json
from datetime import datetime

# Initialize DynamoDB client
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('bike_share_data')  # Replace with your actual table name

def lambda_handler(event, context):
    # Prepare data for DynamoDB
    item = {
        'timestamp': str(datetime.utcnow()),  # Current timestamp
        'last_updated': free_bike_status['last_updated'],
        'ttl': free_bike_status['ttl'],
        'version': free_bike_status['version'],
        'data': {
            'bikes': df_free_bikes.to_dict(orient='records'),
            'vehicle_types': df_vehicle_types.to_dict(orient='records')
        }
    }

    # Write item to DynamoDB
    try:
        table.put_item(Item=item)
        return {
            'statusCode': 200,
            'body': json.dumps('Item written to DynamoDB successfully')
        }
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps(f'Error writing to DynamoDB: {str(e)}')
        }

In [13]:
data = {}
data[free_bike_status['last_updated']] = free_bike_status['data']

In [14]:
import json
import boto3
from datetime import datetime

# Initialize DynamoDB client
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('test_bike_share_data')  # Replace with your actual table name

def make_request(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
        return json.loads(response.text)
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

def lambda_handler(event, context):
    # Example data (could also be from an API call)
    #free bike status
    free_bike_status = make_request("https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/free_bike_status.json")
    data = {}
    data[free_bike_status['last_updated']] = free_bike_status['data']

    # Write item to DynamoDB
    try:
        table.put_item(Item=data)
        return {
            'statusCode': 200,
            'body': json.dumps('Item written to DynamoDB successfully')
        }
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps(f'Error writing to DynamoDB: {str(e)}')
        }


{1750639986: {'bikes': [{'lon': -76.985162854,
    'is_disabled': 0,
    'vehicle_type_id': '2',
    'lat': 38.86598146,
    'is_reserved': 0,
    'current_range_meters': 18990.2592,
    'bike_id': 'dcc06775bf5af87d0be842c4182606e5',
    'rental_uris': {'android': 'https://dc.lft.to/lastmile_qr_scan',
     'ios': 'https://dc.lft.to/lastmile_qr_scan'}},
   {'lon': -77.015416622,
    'is_disabled': 0,
    'vehicle_type_id': '2',
    'lat': 38.974904656,
    'is_reserved': 0,
    'current_range_meters': 5471.7696,
    'bike_id': '5cd1e5f370f4cb336dfb498856707b19',
    'rental_uris': {'android': 'https://dc.lft.to/lastmile_qr_scan',
     'ios': 'https://dc.lft.to/lastmile_qr_scan'}},
   {'lon': -77.023230076,
    'is_disabled': 0,
    'vehicle_type_id': '2',
    'lat': 38.957342148,
    'is_reserved': 0,
    'current_range_meters': 24462.0288,
    'bike_id': 'fef8c8a3568d0e33981757f3e398effc',
    'rental_uris': {'android': 'https://dc.lft.to/lastmile_qr_scan',
     'ios': 'https://dc.lft.

In [27]:
import json
from datetime import datetime
import urllib.request
from decimal import Decimal


# Initialize DynamoDB client


def make_request_urllib(url):
    try:
        with urllib.request.urlopen(url) as response:
            data = response.read()
            return json.loads(data.decode('utf-8'))
    except urllib.error.URLError as e:
        print(f"An error occurred: {e}")
        return None

# Example data (could also be from an API call)
#free bike status
free_bike_status = make_request_urllib("https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/free_bike_status.json")
#data = {'api_name':'free_bikes',
#        'timestamp': free_bike_status['last_updated'],
#        'data':free_bike_status['data']}
data = {
    'api_name': 'free_bikes',
    'timestamp': str(datetime.utcnow()),  # Current timestamp
    'last_updated': free_bike_status['last_updated'],
    #'ttl': free_bike_status['ttl'],
    #'version': free_bike_status['version'],
    'data': free_bike_status['data'],
}
print(data)
#ensure all data that is in float type is decimal
for bike in data['data']['bikes']:
    bike['lat'] = Decimal(bike['lat'])
    bike['lon'] = Decimal(bike['lon'])
    #bike['bike_id'] = int(bike['bike_id'])
    bike['is_reserved'] = int(bike['is_reserved'])
    bike['is_disabled'] = int(bike['is_disabled'])
    bike['vehicle_type_id'] = int(bike['vehicle_type_id'])
# Write item to DynamoDB

{'api_name': 'free_bikes', 'timestamp': '2025-06-23 02:06:27.823848', 'last_updated': 1750644306, 'data': {'bikes': [{'lon': -76.985174298, 'is_disabled': 0, 'vehicle_type_id': '2', 'lat': 38.865972519, 'is_reserved': 0, 'current_range_meters': 18990.2592, 'bike_id': 'eb5b436c4bd87285702787d392060b23', 'rental_uris': {'android': 'https://dc.lft.to/lastmile_qr_scan', 'ios': 'https://dc.lft.to/lastmile_qr_scan'}}, {'lon': -77.014913201, 'is_disabled': 0, 'vehicle_type_id': '2', 'lat': 38.975594878, 'is_reserved': 0, 'current_range_meters': 5471.7696, 'bike_id': '4d16bb5d121cc1ac6052e3ba552cfe0e', 'rental_uris': {'android': 'https://dc.lft.to/lastmile_qr_scan', 'ios': 'https://dc.lft.to/lastmile_qr_scan'}}, {'lon': -77.03077364, 'is_disabled': 0, 'vehicle_type_id': '2', 'lat': 38.922469735, 'is_reserved': 0, 'current_range_meters': 12231.0144, 'bike_id': '91a7391de71db943b21684cbf3c4e830', 'rental_uris': {'android': 'https://dc.lft.to/lastmile_qr_scan', 'ios': 'https://dc.lft.to/lastmile_

In [28]:
Decimal(1)

Decimal('1')